In [1]:
! pip install driftpy

In [2]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
pd.options.plotting.backend = "plotly"

In [3]:
drift_configs = {
    'devnet': {
        'ENV': 'devnet',
        'URL': 'https://api.devnet.solana.com/',
        'IDL_URL':'https://raw.githubusercontent.com/drift-labs/protocol-v1/master/sdk/src/idl/clearing_house.json',
		'PYTH_ORACLE_MAPPING_ADDRESS': 'BmA9Z6FjioHJPpjT39QazZyhDRUdZy2ezwx4GiDdE2u2',
		'CLEARING_HOUSE_PROGRAM_ID': 'AsW7LnXB9UA1uec9wi9MctYTgTz7YH9snhxd16GsFaGX',
		'USDC_MINT_ADDRESS': '8zGuJQqwhZafTah7Uc7Z4tXRnguqkn5KLFAP8oV6PHe2',
	},
    'devnet-limits': {
        'ENV': 'devnet',
        'URL': 'https://api.devnet.solana.com/',
		'PYTH_ORACLE_MAPPING_ADDRESS': 'BmA9Z6FjioHJPpjT39QazZyhDRUdZy2ezwx4GiDdE2u2',
		'CLEARING_HOUSE_PROGRAM_ID': 'HiZ8CnfEE9LrBZTfc8hBneWrPg1Cbsn8Wdy6SPLfae9V',
		'USDC_MINT_ADDRESS': '8zGuJQqwhZafTah7Uc7Z4tXRnguqkn5KLFAP8oV6PHe2',
	},
	'mainnet': {
		'ENV': 'mainnet-beta',
        'URL': 'https://api.mainnet-beta.solana.com/',
        # 'IDL_URL':'',
		'PYTH_ORACLE_MAPPING_ADDRESS': 'AHtgzX45WTKfkPG53L6WYhGEXwQkN1BVknET3sVsLL8J',
		'CLEARING_HOUSE_PROGRAM_ID': 'dammHkt7jmytvbS3nHTxQNEcP59aE57nxwV21YdqEDN',
		'USDC_MINT_ADDRESS': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
	},
}

In [4]:
from solana.publickey import PublicKey
from anchorpy import Idl, Program, Provider
from driftpy.clearing_house import ClearingHouse
from driftpy import idl

import os
import json
import requests

def load_program(ENV):
    CH_PID = drift_configs[ENV].get('CLEARING_HOUSE_PROGRAM_ID')
    IDL_JSON = None
    IDL_URL = drift_configs[ENV].get('IDL_URL', None)
    if IDL_URL is None:
        IDL_JSON = ClearingHouse.local_idl()
    else:
        print('requesting from', IDL_URL)
        IDL_JSON =  Idl.from_json(requests.request("GET", IDL_URL).json())
        
    if "ANCHOR_PROVIDER_URL" not in os.environ:
        os.environ["ANCHOR_PROVIDER_URL"] =  drift_configs[ENV].get('URL')

    # override path to wallet
    os.environ["ANCHOR_WALLET"] = os.path.expanduser("~/.config/solana/arbJ.json") 
    #del os.environ["ANCHOR_WALLET"]    

    # Address of the deployed program.
    program = Program(IDL_JSON,  PublicKey(CH_PID), Provider.env())
    return program

program = load_program('devnet')

requesting from https://raw.githubusercontent.com/drift-labs/protocol-v1/master/sdk/src/idl/clearing_house.json


In [5]:
from driftpy.clearing_house import ClearingHouse

drift_acct = await ClearingHouse.create(program)

# check if user account exists
try:
    drift_user_acct = await drift_acct.get_user_account()
except Exception as e:
    print('Error:', e)

In [6]:
balance = (drift_user_acct.collateral/1e6)
balance

9190181.955206

# load predicted funding

In [7]:
from driftpy.constants.markets import MARKETS
from driftpy.constants.numeric_constants import MARK_PRICE_PRECISION
from datetime import datetime, timedelta

markets = await drift_acct.get_markets_account()

def calculate_market_summary(markets):
    
    FUNDING_PRECISION = 1e4
    
    markets_summary = pd.concat([
        pd.DataFrame(MARKETS).iloc[:,:3],
    pd.DataFrame(markets.markets),
    pd.DataFrame([x.amm for x in markets.markets]),           
              ],axis=1).dropna(subset=['symbol'])

    last_funding_ts = pd.to_datetime(markets.markets[0].amm.last_funding_rate_ts*1e9)
    next_funding_ts = last_funding_ts + timedelta(hours=1)
    next_funding_ts

    summary = {}
    summary['next_funding_rate'] = (markets_summary['last_mark_price_twap'] \
                         - markets_summary['last_oracle_price_twap'])\
    /markets_summary['last_oracle_price_twap']/24
    
    summary['mark_price'] = (markets_summary['quote_asset_reserve'] \
                         /markets_summary['base_asset_reserve'])\
    *markets_summary['peg_multiplier']/1e3
    
    return pd.concat([pd.DataFrame(MARKETS).iloc[:,:3], pd.DataFrame(summary)],axis=1)

In [8]:
calculate_market_summary(markets)

,symbol,base_asset_symbol,market_index,next_funding_rate,mark_price
0,SOL-PERP,SOL,0,0.000265,98.649322
1,BTC-PERP,BTC,1,0.000121,38594.990784
2,ETH-PERP,ETH,2,0.000147,2693.709768
3,LUNA-PERP,LUNA,3,0.000312,51.544529
4,AVAX-PERP,AVAX,4,0.000321,70.267824
5,BNB-PERP,BNB,5,0.000257,386.329275
6,MATIC-PERP,MATIC,6,0.015457,2.098989
7,ATOM-PERP,ATOM,7,0.004160,32.415746
8,DOT-PERP,DOT,8,0.003658,20.658505


# todo: load oracle

In [9]:
!pip install pythclient

In [10]:
from pythclient.pythclient import PythClient  # noqa
from pythclient.ratelimit import RateLimit  # noqa
from pythclient.pythaccounts import PythPriceAccount  # noqa
from pythclient.utils import get_key # noqa

v2_first_mapping_account_key = get_key("mainnet", "mapping")
v2_program_key = get_key("mainnet", "program")

pythC = PythClient(
        first_mapping_account_key=v2_first_mapping_account_key,
        program_key=None, #v2_program_key,
    ) 

await pythC.refresh_all_prices()
        # await c.refresh_all_prices()
        # products = await c.get_products()
        # all_prices: List[PythPriceAccount] = []
        # for p in products:
        #     print(p.key, p.attrs)
        #     prices = await p.get_prices()
        #     for _, pr in prices.items():
        #         all_prices.append(pr)
        #         print(
        #             pr.key,
        #             pr.product_account_key,
        #             pr.price_type,
        #             pr.aggregate_price,
        #             "p/m",
        #             pr.aggregate_price_confidence_interval,
        #         )